In [3]:
import requests as req
from bs4 import BeautifulSoup
import json
import pandas as pd
from standardize import *
from key import api_key
from key import g_api_key
import re
import os
import bokeh as bo
import numpy as np 
import matplotlib as mp
from helper import *
import geopandas as gp
from shapely.geometry import Point

In [4]:
ln_list = ['rats','rat','mice','mouse',\
'racoon','racoons','possum','vermin','rodent','rodents','roach','roaches','cockroach','cockroaches','termite','termites']

url_sql = "https://data.milwaukee.gov/api/3/action/datastore_search?resource_id"

rats_endpoint = "resource_id=bf2b508a-5bfa-49da-8846-d87ffeee020a&q=rats"

historical = 'bf2b508a-5bfa-49da-8846-d87ffeee020a'
current = 'abdfe983-e856-40cd-bee2-85e78454344a'

df = pd.DataFrame()  #pull_data(url_sql, rats_endpoint, api_key)

for rec_id in [historical, current]:
    for rodent_end in ln_list:
        rodent_endpoint = '='+rec_id+'&q='+rodent_end
        new_df = pull_data(url_sql,rodent_endpoint,api_key)
        new_df['lookup_term'] = rodent_end
        df = df.append(new_df)

#df.drop_duplicates(inplace=True)

offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset0
True
offset50000
True
offset0


/Users/ayaspan/anaconda3/envs/mke_calls/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True
offset0
True
offset50000
True


In [5]:
df.shape

(2646, 8)

In [6]:
df.iloc[18].CASECLOSUREREASONDESCRIPTION

''

In [7]:
df.head()

,CASECLOSUREREASONDESCRIPTION,CLOSEDDATETIME,CREATIONDATE,OBJECTDESC,TITLE,_id,lookup_term,rank
0,,2011-11-16 08:15:26,2011-10-26 20:27:57,3820 N HUMBOLDT BL,Rats Or Rat Harborage,1496343.0,rats,0.057309
1,Record number 3164787 is a request for Sanitat...,2018-11-27 10:31:13,2018-11-20 15:05:54,1115 E WRIGHT ST,Tree Trimming Web,16955.0,rats,0.057309
2,Record number 3162955 is a request for Sanitat...,2018-11-19 11:20:37,2018-11-15 13:11:44,5910 N 37TH ST,Call Back Sanitation Web,19222.0,rats,0.057309
3,Record number 3154445 is a request for Sanitat...,2018-11-09 09:46:37,2018-10-26 11:19:48,1337 N 55TH ST,Cart Requests - Garbage Web,29264.0,rats,0.057309
4,Record number 3149741 is a request for Sanitat...,2018-10-18 10:05:35,2018-10-16 11:58:51,3835 N 79TH ST,Cart Requests - Garbage Web,35181.0,rats,0.057309


In [8]:
cols = list(df.columns)
cols.remove('lookup_term')

In [9]:
df = df.groupby(cols).apply(lambda x: ','.join(x.lookup_term)).to_frame().reset_index().rename(columns={'0':'cats'})

In [10]:
df.rename(columns={0:'categories'},inplace=True)

In [11]:
df.head()

,CASECLOSUREREASONDESCRIPTION,CLOSEDDATETIME,CREATIONDATE,OBJECTDESC,TITLE,_id,rank,categories
0,,2011-11-16 08:15:26,2011-10-26 20:27:57,3820 N HUMBOLDT BL,Rats Or Rat Harborage,1496343.0,0.057309,"rats,rat"
1,,2011-12-01 08:15:28,2011-11-24 19:03:40,2123 N 47TH ST,Rats Or Rat Harborage,1495292.0,0.057309,"rats,rat"
2,,2011-12-01 08:15:29,2011-11-24 19:08:25,4712 W LLOYD ST,Rats Or Rat Harborage,1495290.0,0.057309,"rats,rat"
3,,2012-02-20 09:59:58,2011-12-24 23:41:44,2904 W WISCONSIN AV # 101,Rats Or Rat Harborage,1493156.0,0.057309,"rats,rat"
4,,2012-02-21 08:16:54,2012-02-02 18:17:51,4712 W LLOYD ST,Rats Or Rat Harborage,1486784.0,0.057309,"rats,rat"


In [12]:
titles_df = df.groupby('TITLE').size().to_frame().reset_index().rename(columns={0:'cts','TITLE':'issue'}).sort_values('cts', ascending=False)
tt_5 = titles_df[:5]

In [13]:
categories_df = df.groupby('categories').size().to_frame().reset_index().rename(columns={0:'cts'}).sort_values('cts', ascending=False)

In [14]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource
from math import pi


output_notebook()
#output_file('vbar.html')

#data = {'issues': list(tt_5.issues), 'cts': np.ndarray(tt_5.cts)}

source = ColumnDataSource(data=dict(issue=tt_5.issue, cts=np.array(tt_5.cts)))

issue = source.data['issue'].tolist()

#print(dict(issues=list(tt_5.issue), cts=np.array(tt_5.cts)))

p = figure(x_range=tt_5.issue,plot_width=400, plot_height=400,toolbar_location=None)
p.vbar(x='issue', width=.9, bottom=0, source = source,
        top='cts', fill_color=factor_cmap('issue', palette=Spectral6, factors=tt_5.issue))
p.xaxis.major_label_orientation = pi/2.7
#p.vbar(x =titles_df.issue, width=0.5, top = titles_df.cts, bottom=0,color="firebrick")
show(p)


Loading BokehJS ...

In [15]:
df[df.TITLE=='Mice/Roach Infestation']

,CASECLOSUREREASONDESCRIPTION,CLOSEDDATETIME,CREATIONDATE,OBJECTDESC,TITLE,_id,rank,categories


In [17]:
tc_5 = categories_df[:5]

source = ColumnDataSource(data=dict(categories=tc_5.categories, cts=np.array(tc_5.cts)))

issue = source.data['categories'].tolist()

p = figure(x_range=tc_5.categories, plot_width=400, plot_height=400,toolbar_location=None)
p.vbar(x='categories', width=.9, bottom=0, source = source,
        top='cts', fill_color=factor_cmap('categories', palette=Spectral6, factors=tc_5.categories))
p.xaxis.major_label_orientation = pi/2.7
#p.vbar(x =titles_df.issue, width=0.5, top = titles_df.cts, bottom=0,color="firebrick")
show(p)

In [51]:
#c(" AV ", " DR ", " ST ", " RD ", " BL ", " PL ",
#                    " LA ", " CR ", " WA ", " CT ", " TR ", " PK ")
#http://maps2.milwaukee.gov/ArcGIS/rest/services/geocode/MAIthenDIME_geocode/GeocodeServer/findAddressCandidates?Street=3820+North+Humboldt+BL&SingleLine=&outFields=Loc_name&outSR=&f=json
import json
from bs4 import BeautifulSoup

def geo_locate(x):
    if not len(x):
        return [None,None,None]
    gl = "http://maps2.milwaukee.gov/ArcGIS/rest/services/geocode/MAIthenDIME_geocode/GeocodeServer/findAddressCandidates?Street="
    address = x.replace(' ','+')
    response = req.get(gl+address+'&SingleLine=&outFields=Loc_name&outSR=&f=json')
    if 'html' in str(response.content):
        soup = BeautifulSoup(response.content,'html.parser')
        #print(soup)
        #for i in soup.find('ul'):
        east, north = re.findall(r'[0-9/.]+',soup.find('ul').text)
        zipcode  = re.findall(r'[0-9]{5}',soup.find('ul').text)[0]
        if zipcode !=None:
            return [east,north,zipcode]
        else:
            return [east,north,None]
    else:
        #print(address)
        #print(response.content)
        res = response.json()
        if not len(res['candidates']):
            return [None,None,None]
        #print(res)
        result = res['candidates'][0]
        #print(result)
        geodata = dict()
        geodata['east'] = result['location']['x']
        geodata['north'] = result['location']['y']
        
        result1 = res['candidates'][1]
        
        geodata['address'] = result1['address']
        
        #print(geodata['address'])
        
        if (len(geodata['address'].split(',')) == 2):
        
            geodata['zip'] = geodata['address'].split(',')[1]
            
        else:
            
            geodata['zip'] = None
            
        #print(geodata['address'])
        
    return [geodata['east'], geodata['north'], geodata['zip']]

In [52]:
df['utm_zip'] = df.OBJECTDESC.apply(lambda x: geo_locate(x))

In [54]:
df.utm_zip.tail()

2550    [2562623.6858014404, 363739.24942488456,  53207]
2551     [2532781.4447490927, 415529.0855607738,  53218]
2552     [2547794.732782273, 392340.90181008715,  53208]
2553     [2536159.739935282, 366801.42942045926,  53219]
2554      [2540418.631161882, 391224.9931794932,  53208]
Name: utm_zip, dtype: object

In [55]:
df['easting'] = df.utm_zip.apply(lambda x: float(x[0]) if x[0] !=None else None)
df['northing'] = df.utm_zip.apply(lambda x: float(x[1]) if x[1] !=None else None)
df['point'] = df.utm_zip.apply(lambda x: Point([float(x[0]),float(x[1])]) if x[1] !=None else None)
df['zip'] = df.utm_zip.apply(lambda x: x[2] if x[2] !=None else None)

IndexError: list index out of range

In [ ]:
df.drop(['lat_long'], axis=1, inplace=True)
df.head()

In [ ]:
df.to_csv('rodents.csv')

In [ ]:
mil_link= 'neighborhood2.shp'
mil = gp.read_file(mil_link)
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm

#colors
#https://matplotlib.org/gallery/color/colormap_reference.html

#cmap = ListedColormap(['winter'], name='winter')

fig,ax = plt.subplots(1, 1, figsize=(20,10))
mil.plot(color='darkgrey', ax =ax)

event_locs = gp.GeoDataFrame(df[['point']],
                            crs=mil.crs).rename(columns={'point':'geometry'})

event_locs.plot(ax=ax, 
                                color = 'blue', 
                                marker = '*',
                                markersize = 60)

ax.set(xlabel = 'Easting',
      ylabel='Northing',
      title="Milwaukee Rodents")
ax.set_axisbelow(True)
ax.yaxis.grid(color ='gray',
             linestyle='dashed')
ax.xaxis.grid(color ='gray',
             linestyle='dashed')
